
# Final Project

In [37]:
import pandas as pd
import time
import re
import datetime
from datetime import date
from datetime import datetime
from pandas.io.json import json_normalize

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


# 1) Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.


In [2]:
start = time.time()
df_loans_lenders = pd.read_csv("C:/Users/Filippo/Desktop/loans_lenders.csv")

df_loans_lenders.head()


,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."


In [3]:
df_loans_lenders_norm = (df_loans_lenders.set_index(['loan_id'])
   .apply(lambda x: x.str.split(',').explode())
   .reset_index())

df_loans_lenders_norm.head()

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499


# 2) For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.


In [4]:
df_loans = pd.read_csv("C:/Users/Filippo/Desktop/loans.csv")


df_loans.head()



,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,loan_use,country_code,country_name,town_name,currency_policy,currency_exchange_coverage_rate,currency,partner_id,posted_time,planned_expiration_time,disburse_time,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,to buy additional inventories for her sari-sa...,PH,Philippines,Ozamiz - Ozamiz City,shared,0.1,PHP,126.0,2014-01-15 02:23:45.000 +0000,2014-02-14 03:30:06.000 +0000,2013-12-22 08:00:00.000 +0000,2014-01-15 04:48:22.000 +0000,7.0,3,2,1,NaN,female,true,irregular,field_partner
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,"To buy American clothing such as blouses, shir...",HN,Honduras,"La Lopez, Choloma",shared,0.1,HNL,201.0,2014-01-14 20:23:20.000 +0000,2014-03-26 22:25:07.000 +0000,2013-12-20 08:00:00.000 +0000,2014-02-25 06:42:06.000 +0000,8.0,11,2,1,NaN,female,true,monthly,field_partner
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,"to buy stock of rice, sugar and flour",PK,Pakistan,Lala Musa,shared,0.1,PKR,245.0,2014-01-16 11:32:58.000 +0000,2014-02-15 21:10:05.000 +0000,2014-01-09 08:00:00.000 +0000,2014-01-24 23:06:18.000 +0000,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,"to buy cucumber and tomato seeds, as well as f...",KG,Kyrgyzstan,"Aravan village, Osh region",shared,0.1,KGS,171.0,2014-01-20 09:59:48.000 +0000,2014-02-21 03:10:02.000 +0000,2014-01-17 08:00:00.000 +0000,2014-01-22 05:29:28.000 +0000,14.0,21,2,1,user_favorite,female,true,monthly,field_partner
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,to buy organic fertilizer and agrochemical pr...,PH,Philippines,"Baleleng, Sto. Thomas, Isabela",shared,0.1,PHP,123.0,2014-01-14 05:46:21.000 +0000,2014-02-13 06:10:02.000 +0000,2013-12-17 08:00:00.000 +0000,2014-01-14 17:29:27.000 +0000,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner


In [5]:
df_loans['planned_expiration_time'] = pd.to_datetime(df_loans['planned_expiration_time'])
df_loans['disburse_time'] = pd.to_datetime(df_loans['disburse_time'])
df_loans['duration'] = (df_loans['planned_expiration_time'] - df_loans['disburse_time']).apply(lambda x: x.days)

df_loans['duration']

0          53.0
1          96.0
2          37.0
3          34.0
4          57.0
           ... 
1419602    39.0
1419603    39.0
1419604    51.0
1419605    63.0
1419606    61.0
Name: duration, Length: 1419607, dtype: float64

# 3) Find the lenders that have funded at least twice.

In [6]:
df_lenders_and_loans = pd.merge(df_loans_lenders_norm, df_loans[['loan_id','status','loan_amount']], on = "loan_id")
loan_funded = df_lenders_and_loans.loc[df_lenders_and_loans['status'].isin(['funded'])]

loan_funded['lenders'].value_counts().loc[lambda x: x >= 2]      

 gooddogg1             142349
 trolltech4460         139191
 gmct                  122156
 nms                    93283
 themissionbeltco       70151
                        ...  
 wendy8302                  2
 glaucia2545                2
 robertplantfan2502         2
romwaldorannie4591          2
 mijo3756                   2
Name: lenders, Length: 993782, dtype: int64

# 4) For each country, compute how many loans have involved that country as borrowers.

In [7]:
df_loans['country_name'].value_counts()

Philippines         285336
Kenya               143699
Peru                 86000
Cambodia             79701
El Salvador          64037
                     ...  
Botswana                 1
Uruguay                  1
Papua New Guinea         1
Mauritania               1
Canada                   1
Name: country_name, Length: 96, dtype: int64

# 5) For each country, compute the overall amount of money borrowed.

In [8]:
country_borrowed_amount = df_loans.groupby('country_code')['loan_amount'].sum()

country_borrowed_amount

country_code
AF     1967950.0
AL     4307350.0
AM    22950475.0
AZ    14784625.0
BA      477250.0
         ...    
XK     3083025.0
YE     3444000.0
ZA     1006525.0
ZM     1978975.0
ZW     5851875.0
Name: loan_amount, Length: 95, dtype: float64

# 6) Like the previous point, but expressed as a percentage of the overall amount lent.

In [10]:
country_borrowed_amount /df_loans['loan_amount'].sum() *100

country_code
AF    0.166573
AL    0.364586
AM    1.942589
AZ    1.251410
BA    0.040396
        ...   
XK    0.260955
YE    0.291509
ZA    0.085195
ZM    0.167506
ZW    0.495318
Name: loan_amount, Length: 95, dtype: float64

# 7) Like the three previous points, but split for each year (with respect to disburse time)

In [11]:
df_loans.groupby([df_loans['country_name'
                              ],df_loans['disburse_time'].dt.year])['country_name'].value_counts()

country_name  disburse_time  country_name
Afghanistan   2007.0         Afghanistan      408
              2008.0         Afghanistan      370
              2009.0         Afghanistan      678
              2010.0         Afghanistan      632
              2011.0         Afghanistan      247
                                             ... 
Zimbabwe      2013.0         Zimbabwe         426
              2014.0         Zimbabwe        2078
              2015.0         Zimbabwe         600
              2016.0         Zimbabwe         808
              2017.0         Zimbabwe        1079
Name: country_name, Length: 748, dtype: int64

Overall amount of money borrowed per Country, divided for each year.

In [12]:
df_loans.groupby([df_loans['country_name'
                          ],df_loans['disburse_time'].dt.year])['loan_amount'].sum() 

country_name  disburse_time
Afghanistan   2007.0            194975.0
              2008.0            365375.0
              2009.0            585125.0
              2010.0            563350.0
              2011.0            245125.0
                                 ...    
Zimbabwe      2013.0            678525.0
              2014.0           1311575.0
              2015.0            723625.0
              2016.0            788600.0
              2017.0           1237600.0
Name: loan_amount, Length: 748, dtype: float64

Overall amount of money borrowed per Country as percentage, divided for each year.

In [13]:
df_loans.groupby([df_loans['country_name'],
                    df_loans['disburse_time'].dt.year])['loan_amount'].sum() / df_loans['loan_amount'].sum() *100

country_name  disburse_time
Afghanistan   2007.0           0.016503
              2008.0           0.030926
              2009.0           0.049527
              2010.0           0.047683
              2011.0           0.020748
                                 ...   
Zimbabwe      2013.0           0.057432
              2014.0           0.111015
              2015.0           0.061250
              2016.0           0.066749
              2017.0           0.104754
Name: loan_amount, Length: 748, dtype: float64

# 8) For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.

In [14]:
df_loans['lenders_contribute'] = df_loans['loan_amount'] / df_loans['num_lenders_total']

df_lenders_contribute = pd.merge(df_loans_lenders_norm, df_loans[['loan_id'
                                                            ,'lenders_contribute']]
                                 , on = "loan_id")

df_lenders_contribute.groupby('lenders')['lenders_contribute'].sum()

lenders
 000               1485.309656
 00000             1249.947363
 0002              2201.180463
 00mike00            38.461538
 0101craign0101    2424.088932
                      ...     
zzanita              62.500000
zzcyna7269           44.285714
zzinnia              32.758621
zzmcfate           2033.750197
zzrvmf8538          112.820513
Name: lenders_contribute, Length: 1639026, dtype: float64

# 9) For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.

Difference between the overall amount of money lent and the overall amount of money borrowed per Country.

In [15]:
df_lenders = pd.read_csv("C:/Users/Filippo/Desktop/lenders.csv")

df_lenders.head()

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
0,qian3013,Qian,NaN,NaN,NaN,1461300457,NaN,NaN,1.0,NaN,0
1,reena6733,Reena,NaN,NaN,NaN,1461300634,NaN,NaN,9.0,NaN,0
2,mai5982,Mai,NaN,NaN,NaN,1461300853,NaN,NaN,NaN,NaN,0
3,andrew86079135,Andrew,NaN,NaN,NaN,1461301091,NaN,NaN,5.0,Peter Tan,0
4,nguyen6962,Nguyen,NaN,NaN,NaN,1461301154,NaN,NaN,NaN,NaN,0


In [16]:
df_lenders.dropna(subset = ['country_code'], inplace = True)
df_lenders = df_lenders.rename(columns = {'permanent_name': 'lenders'})


df_merge_ll = pd.merge(df_lenders[['lenders'
                                        , 'country_code']]
                        , df_lenders_and_loans 
                        , on = "lenders")

df_merge_ll.head()

,lenders,country_code,loan_id,status,loan_amount
0,naresh2074,US,1059280,funded,700.0
1,naresh2074,US,1055468,funded,625.0
2,prashankpradeep8456,US,1055545,funded,300.0
3,pierremarie5489,FR,1053402,funded,575.0
4,radoslava6314,SK,1060052,funded,300.0


In [17]:
country_distribution = df_merge_ll.groupby('country_code')['loan_amount'].sum() / df_merge_ll['loan_amount'].sum()
country_distribution_amount = country_distribution * df_loans['loan_amount'].sum()
df_amount_per_country = pd.merge(country_distribution_amount, country_borrowed_amount, on = 'country_code')
difference = pd.DataFrame(df_amount_per_country['loan_amount_x'] - df_amount_per_country['loan_amount_y'])
difference = difference.rename(columns = { 0 : 'difference'})

difference.head()

,difference
country_code,
AF,-1.879762e+06
AL,-4.285781e+06
AM,-2.291993e+07
AZ,-1.478048e+07
BA,-3.607563e+05


# 10) Which country has the highest ratio between the difference computed at the previous point and the population?

Country with the highest ratio between the difference computed at Exercise 9 and the population.

In [18]:
df_country_stats = pd.read_csv("C:/Users/Filippo/Desktop/country_stats.csv")

df_country_stats

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name
0,India,IN,IND,Asia,Southern Asia,1339180127,21.9,0.623559,68.322,11.696590,6.298834,5663.474799,India
1,Nigeria,NG,NGA,Africa,Western Africa,190886311,70.0,0.527105,53.057,9.970482,6.000000,5442.901264,Nigeria
2,Mexico,MX,MEX,Americas,Central America,129163276,46.2,0.761683,76.972,13.299090,8.554985,16383.106680,Mexico
3,Pakistan,PK,PAK,Asia,Southern Asia,197015955,29.5,0.550354,66.365,8.106910,5.089460,5031.173074,Pakistan
4,Bangladesh,BD,BGD,Asia,Southern Asia,164669751,31.5,0.578824,71.985,10.178706,5.241577,3341.490722,Bangladesh
...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,Somalia,SO,SOM,Africa,Eastern Africa,14742523,NaN,NaN,NaN,NaN,NaN,NaN,Somalia
170,Central African Republic,CF,CAF,Africa,Middle Africa,4659080,NaN,0.352440,51.458,7.098980,4.230000,587.473961,Central African Republic
171,Samoa,WS,WSM,Oceania,Polynesia,196440,NaN,0.702000,NaN,NaN,NaN,NaN,Samoa
172,Palestine,PS,PS,Asia,Western Asia,4920724,NaN,0.677000,NaN,NaN,NaN,NaN,Palestine


In [20]:
df_merge_final = pd.merge(difference, df_country_stats[['population_below_poverty_line'
                                           , 'country_code'
                                           , 'population']]
                     , on = 'country_code')

df_merge_final = df_merge_final.set_index('country_code')

(df_merge_final['difference'] / df_merge_final['population']).idxmax()

'CA'

# 11) Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?

In [22]:
(df_merge_final['difference'] / df_merge_final['population_below_poverty_line']).idxmax()

'US'